In [1]:
import pandas as pd

df1 = pd.read_csv("data/train_dataset.csv")
df2 = pd.read_csv("data/valid_dataset.csv")
df3 = pd.read_csv("data/test_dataset.csv")

df = pd.concat([df1, df2, df3]).reset_index(drop=True)
df = df.loc[:, "Target Sequence"].drop_duplicates().reset_index(drop=True)
df

0      MLKFKYGARNPLDAGAAEPIASRASRLNLFFQGKPPFMTQQQMSPL...
1      MRHSKRTYCPDWDDKDWDYGKWRSSSSHKRRKRSHSSAQENKRCKY...
2      MRHSKRTHCPDWDSRESWGHESYRGSHKRKRRSHSSTQENRHCKPH...
3      MAPFLRIAFNSYELGSLQAEDEANQPFCAVKMKEALSTERGKTLVQ...
4      MVVFNGLLKIKICEAVSLKPTAWSLRHAVGPRPQTFLLDPYIALNV...
                             ...                        
424    MEKDGLCRADQQYECVAEIGEGAYGKVFKARDLKNGGRFVALKRVR...
425    EESEEEEEEEEEEEEETGSNSEEASEQSAEEVSEEEMSEDEERENE...
426    MAAILGDTIMVAKGLVKLTQAAVETHLQHLGIGGELIMAARALQST...
427    MKRSEKPEGYRQMRPKTFPASNYTVSSRQMLQEIRESLRNLSKPSD...
428    MAEPSGSPVHVQLPQQAAPVTAAAAAAPAAATAAPAPAAPAAPAPA...
Name: Target Sequence, Length: 429, dtype: object

In [2]:
import pickle
import torch
import pandas as pd
import transformers
from tqdm import tqdm
from transformers import AutoModel, BertTokenizer, RobertaTokenizer

prot_tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert_bfd", do_lower_case=False)
prot_encoder = AutoModel.from_pretrained("Rostlab/prot_bert_bfd").to("cuda")
prot_encoder.eval()

/home/resperanca/miniconda3/envs/DLM/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30, 1024, padding_idx=0)
    (position_embeddings): Embedding(40000, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-29): 30 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.0, i

In [4]:
import os
os.makedirs('prot_feat', exist_ok=True)

for max_length in [702]:
    results = {}

    for data in tqdm(df, total=len(df)):
        seq = prot_tokenizer(" ".join(data), max_length=max_length + 2, truncation=True, return_tensors="pt").to("cuda")
        a = prot_encoder(**seq)
        a = a.last_hidden_state.detach().to("cpu")
        results[data[:20]] = a[:, 0]

    with open(f"prot_feat/{max_length}_cls.pkl", "wb") as f:
        pickle.dump(results, f)

100%|██████████| 429/429 [00:25<00:00, 16.91it/s]


In [1]:
import torch

# Limpar a memória da GPU
torch.cuda.empty_cache()
